<a href="https://colab.research.google.com/github/Benendead/LSTMjazz/blob/master/Research/Related_Works.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Source 1: [Text-based LSTM networks for Automatic Music Composition](https://arxiv.org/pdf/1604.05358.pdf)

## Introduction

This paper used text-based LSTMs for music composition. They found that word-RNNs could learn chord progressions and drum tracks, while char-RNNs could only succeed with chord progressions.

Their work attempted to generate jazz chord progressions and rock drum tracks. Their work is notable for two reasons:
1. Their LSTMs were designed to learn from text data as opposed to music symbols or numeric values.
2. They used a larger dataset than previous work, which allows learning of more complex chord progressions.

## Architecture

Char-RNNs are RNNs with character-based learning, as opposed to the conventional word-based learning. These return a vector corresponding to a character, while word-RNNs predict vectors corresponding to unique words. The merits of char-RNNs include:
1. Minimal assumptions are made - there are no constraints on the form of the text representation. Can RNNs learn musical information with such a weak assumption?
2. Fewer characters means fewer states, which reduces computational costs. Their chord vocabulary had 1,259 "words" yet this reduces to 39 characters.

It's worth noting that char-RNNs need to form longer-term dependencies, as the sequence becomes longer. LSTMs help with this, but the trade-off does have downsides we'll see later.

They used two LSTM layers, each with 512 hidden units. According to [reddit](https://www.reddit.com/r/MachineLearning/comments/87djn7/d_what_is_meant_by_number_of_hidden_units_in_an/), this 512 refers to the dimensionality of the hidden state ([Keras documentation](https://keras.io/layers/recurrent/) supports this answer). Dropout of 0.2 was added after each LSTM layer.

The model was put together using *Keras*. Loss function was categorical cross-entropy and optimizer was ADAM. This performed just as well as SGD with Nestrov momentum.

The prediction is stochastic in that each prediction for time $t$, the network outputs probabilities for every state. The system also had a diversity parameter $\alpha$ in the prediction stage, which suppresses $(\alpha < 1)$ or encourages $(\alpha > 1)$ the diversity of prediction by re-weighting the probabilities. This is done using the formula:  
$\hat{p}_i=e^{log(p_i)/\alpha}$, where $p_i$ is the probability for the $i$ states.  
Finally, one of the states is selected using the re-weighted probabilities.

The same size and number of layers was kept for both the char- and word-RNN experiments. The [code](https://github.com/keunwoochoi/lstm_real_book) is available online. I'll look through it once I'm done with the paper.

## Case Study: Chord Progressions

### Representation

They avoid vector representations of the chords and instead use only the text representations. They filled in each chord for all quarter notes during the chord and that's the dataset. Ex:  

F:9 F:9 F:9 F:9 D:min7
D:min7 G:9 G:9 C:maj C:maj
F:9 F:9 C:maj C:maj C:maj
C:maj

Their dataset included 2,486 scores from real and fake books. They transposed everything to C and filled in the duration of chords per quarter note. All song data was appended using _START_ and _END_ flags.

Even when transposed to C, only 867 scores end in C:maj, 489 in G:7, 186 C:maj6, 52 F:maj, and 1,252 in other chords. There were 1,259 unique chords in the training data, which gave the word-RNN a vocab size of 1,259. There were 39 distinct characters, 539,609 chords, and 3,531,261 characters total.

### Results

The system was set to output a chord progression for each diversity parameter after every iteration. Both the word- and char-RNNs showed well structured results. They learned local structures of chords and bars as well as the _START_ and _END_ tags.

After enough training, both results showed chord progressions from real jazz grammar. In  char-RNN, this included ii-V-I, passing chords, modal interchange chords, and tritone substitutions (B:7 for F:7). For word-RNN, these included modal interchanges, circle of fifths (Eb:sus - Gb:maj6 - B:maj7), and descending bass.

The approaches differed slightly in output, as the word-RNN appeared to prefer more conventional progressions than in char-RNN. This may be caused by the different effective lengths of the approaches, as the char-RNN effective has a shorter memory span.

## Conclusion

They used text-based LSTMs to generate chord progressions based on jazz real books. Both word-RNNs and char-RNNs worked in this use case. Their usage of a diversity parameter gives composers a useful tool to dial in desired effects.

# Source 2: [Charlie Parker's Omnibook Data](https://members.loria.fr/KDeguernel/omnibook/)

Just a note that this is the dataset we used. Citation below:

Using Multidimensional Sequences For Improvisation In The OMax Paradigm
Ken Déguernel, Emmanuel Vincent, Gérard Assayag
13th Sound and Music Computing Conference, Aug 2016, Hamburg, Germany. 〈http://quintetnet.hfmt-hamburg.de/SMC2016/〉

# Source 3: [A First Look at Music Composition using LSTM Recurrent Neural Networks](http://people.idsia.ch/~juergen/blues/IDSIA-07-02.pdf)

**Abstract**  
Music composed by RNNs typically suffers from a lack of global structure. Note-by-note transitions or phrase reproduction might succeed, but overall musical form eludes RNNs' grasp. Luckily, LSTMs can overcome the long-term dependency issues and create music that's surprisingly pleasing in the blues style.

## 1. Introduction

A most basic attempt a composition with RNNs might consider single-step predictions. This can then be seeded with just one example and attempt to compose from there. To state the obvious, feed-forward networks coud never perform such a task, as they can't store information about the past. RNNs overcome this basic limitation, at least.

RNNs still struggle with this task, unfortunately. Mozer, in 1994, wrote of his RNN's compositions that “While the
local contours made sense, the pieces were not musically coherent, lacking thematic structure and having minimal phrase structure and rhythmic organization." This problem likely links to vanishing gradients, as in both BPTT and RTRL error flow tends to explode or vanish.

Of course music requires this overall contour Mozer mentions. At the most basic form, early rock-and-roll might consist of four-bar phrases which easily become 32 events or more as the time step is defined as eighth notes. Before this paper, Mozer's single-note melodies were the most relevant work. Mozer had used RNNs with BPTT, probabilistic output values, and a psychologically realistic encoding that gave bias towards chromatically and harmonically related notes. He also encoded events in fewer time steps by treating all notes as one time step. Even still, the architecture failed to capture global musical structure.

Mozer suggested that for a note-by-note method to work, it requires a network which can induce structure at multiple levels. This paper offers that network.



## 2. An LSTM Music Composer

### LSTM Overview

To summarize LSTMs, they're designed to obtain constant error flow through time so that nothing explodes or vanishes. LSTMs use linear units called Constant Error Carousels (CECs), apparently. Really the CEC is just another way to represent the continuing cells we otherwise get when we consider time steps.

The gates which control information into the CEC include:
1. Multiplicative **Input Gate** - Learns to protect the information passed into the CEC by rejecting irrelevant inputs.
2. Multiplicative **Output Gate** - Learns to protect other areas from currently irrelevant memory contents of the CEC.
3. **Forget Gate** - Learns to reset memory cells when their content is obsolete.

Learning is then done using a modified BPTT and a customized version of RTRL. That is, output units use BPTT, output gates use truncated BPTT, and the input and forget gates use truncated RTRL.

### Data Representation

This paper represented the data in simple local form, using one input per note, with 1 as on and 0 as off. In later experiments, they adjusted input units to have a mean of 0 and a standard deviation of 1 (???). This left it to the network to develop a bias towards chromatic or harmonic notes. Their reasons follow:
1. Implicitly multivoice and makes no distinction between chords and melodies. (They implemented chords simply by including them in the single input vector)
2. It's easy to generate probability distributions over the set of possible notes, as they can treat single notes as independent or dependent.

Time was represented as a single input vector representing one slice of real time. The stepsize of each time step of course can vary; they chose an eighth note as theirs. This is preferable for LSTMs, as the network needs to learn relative durations of notes as to create rhythm and counting.

This representation does ignore two issues:
1. There's no indication where a note ends. Thus eight eighth notes in a row of the same notes are equivalent to four quarters or a whole note. One way to overcome this could be to decrease the stepsize of quantization and mark note endings with a zero (???).
2. A second method was suggested by [Todd in 1989](https://pdfs.semanticscholar.org/81e0/a57abde1bf2cc4b7cd772e0573e92069e8ef.pdf). This used special units in the network to indicate the beginning of notes. These authors were unsure how that might scale to multi-voice melodies.

The LSTM was given a range of 12 notes for the chords and 13 notes for melodies. These were simply two sections of the vector but otherwise were represented no differently.  
![Notes Possible](https://i.imgur.com/EqkNFCU.png)

### Training Data

This experiment used a form of 12-bar blues with a quantization step of 8 notes per bar. Each song was thus 96 time steps long. The same chords were used in every song, and they were inverted as to fit into the 12 note options for the chords. Experiment 1 used only the chords whereas Experiment 2 also included a single melody line based on the pentatonic scale.

Training melodies were constructed by concatenating bar-long segments of music by the first author to fit each chord. The datasets were then constructed by concatenating random samples from the $(n=2^{12}=4096)$ possible songs. The melodies included no rests and were only quarter notes.

## 3. Experiment 1 - Learning Chords

Can an LSTM reproduce a musical chord structure? The motivation here is to ensure that the LSTM doesn't require melody.

### Network Topology and Hyperparameters

The chords used were the same 12-bar blues. The network had four cell blocks containing 2 cells each, fully connected to each other and to the input layer. The output layer was fully connected to all cells and the input layer. Forget, input, and output gate biases for the four blocks were set to -0.5, -1.0, -1.5, and -2.0. This causes the blocks to come online one by one. Output biases were set to 0.5, the learning rate 0.00001, and momentum rate 0.9.

Weights were burned after every timestep, as experiments showed that learning was faster if the network was reset after making one or a few gross errors. Resetting went as follows:  
On error, burn existing weights, reset input pattern and clear partial derivatives, activations, and cell states. This was similar to Gers and Schmidhuber's (2000) approach. The output function was the logistic sigmoid with range [0,1].

### Training/Testing

The goal was to predict the probability for a given note to be on or off. For predicting probabilities, RMSE is not appropriate and thus the network was trained using cross-entropy as the objective function. The error function $E_i$ for output activation $y_i$ and target $t_i$ was:  
$E_i=-t_i*ln(y_i)-(1-t_i)*ln(1-y_i).$

This gives a $\delta$ term at the output layer of $(t_i-y_i)$. See [Joost and Schiffmann](https://www.worldscientific.com/doi/abs/10.1142/S0218488598000100) (1998) for what exactly this means. Note that they treat the outputs as statistically inpedendent of one another. Even if this assumption is untrue, it allows the network to predict chords and melodies in parallel (as well as multi-voice melodies).

The testing started the network with the inputs from the first timestep and used network predictions for ensuing time steps. The decision threshold for chord notes was 0.5. Training ended once the network spit back the correct chord sequence in entirety.

### Results

In the end, the network could accomplish this task with a variety of learning rates and momentum rates. The network was able to generate continuing cycles of the progression as well. This result is not surprising, honestly. The learning time took anywhere from 15 to 45 minutes on a 1Ghz Pentium.

## 4. Experiment 2 - Learning Melody and Chords

In this experiment, both parts were learned. They continued learning until the chord structure was learned and cross-entropy was relatively low. Once learning ended, the network was seeded with a note or a series of notes and then allowed to compose freely. The goal was to test if LSTMs could learn chord/melody structure and then use that when composing new songs.

### Network Topology and Hyperparameters

In this case, some cell blocks processed chord information and some melody information. Eight cell blocks with two cells each were used. Four blocks were fully connected to the inputs for melody. The chord cell blocks had recurrent connections to themselves and the melody blocks, whereas the melody blocks were only recurrently connected to other melody blocks. Another way to say this is that melody information never reaches chord cell blocks.

At the output layer, output units for chords are fully connected to chord blocks as well as chord inputs. The melody outputs were fully connected to melody blocks and melody inputs. This is discussed in Section 5.

Forget gate, input gate, and output gate biases were set to -0.5, -1.0, -1.5, and -2.0 for the chord and melody blocks. All other parameters were identical to Experiment 1.

### Training/Testing

The goal was to predict the probability a given note to be on or off. For chords, the same 0.5 decision threshold was used. The melody was restricted to choosing one note per time step. This is done by making the melody outputs sum to 1 and then using a uniform random number in [0,1] to pick the next note. Again, this is discussed in Section 5.

The network was trained until it learned the chord structure and objective error plateaued. The network was then used to compose music by providing a single note or series of notes and then presenting the network outputs as inputs to the next time step. No algorithmic or statistical method was used to evaluate the output music's quality.

### Results

The LSTM indeed composed blues music. It fully learned the chord structure and then used that to constrain its melody output. They urge the reader to visit [their site](https://people.idsia.ch/~juergen/blues/) (Just delete the s in https) to judge the LSTM's music themselves. The network's results do far better than randomly stumbling around the pentatonic scale. One limitation is the fundamental same chord progression, but the compositions are indeed remarkable nonetheless.

## 5. Discussion

The experiments were clearly successful, and to the authors' knowledge, theirs was the first successful use of LSTMs to compose globally coherent music. They acknowledge that more research is needed to see whether the LSTM can deal with more difficult tasks.

### Training Data

The chord structure, for one, was uniform across the training set. Thus this experiment might be more acccurately defined as learning to solo over a predefined form. Their time step was also quite low, as 8 notes per measure is quite easier than, say, 32 time steps per whole note. (My approach plans on 48 steps per whole note)

### Network Architecture

The network's connections were divided between chords and melody, with chords influencing melody but not vice-versa. This choice is justified by the fact that a soloist follows the chord structure supplied by a rhythm section. This choice does presume, though, that we know how to segment input chords from melodies. With jazz sheet music, though, changes are provided separately from melodies and so this is no huge problem. That said, classical music as well as audio signals mix the two.

"Much more research is warranted." Comparing BPTT and RTRL, as well as other RNN variants, would help support a claim about LSTM's relative effectiveness. A more interesting training set might also allow for more interesting compositions. Finally, research circa 2002 led the authors to believe that LSTM works better using a Kalman filter to update weights.

The current architecture is also limited to mere symbolic representations. If it were modified to work with MIDI or audio, it could be used for interactive improvisation. This would require dealing with temporal noise, but research from oscillator beat tracking models (Eck) to LSTM might help calm this noise.

## 6. Conclusion

Their LSTM model successfully learned the blues form and was able to improvise over it. The LSTM could learn the form without melody, and could also learn to compose new melodies along with the chords. "Much more work is warranted." They demonstrated that an RNN can capture local melody structures and long-term structures, which represents an advance for neural network music composition. The site for the paper's results is [here.](http://people.idsia.ch/~juergen/blues/)

-Fin 1/10/2019-

# Source 4: [Learning to Create Jazz Melodies Using Deep Belief Nets](https://www.cs.hmc.edu/~keller/jazz/improvisor/ICCCX-Bickerman-Bosley-Swire-Keller.pdf)

**Abstract** - This paper describes an unsupervised learning technique which automatically creates jazz improvisation over chord sequences. They trained deep belief nets, which are based on restricted Boltzmann machines. They present their encoding scheme, the specifics of learning, and the creation process for their resulting music. Their model created novel jazz licks and should be regarded as a feasibility study for whether such networks could be used at all. Clearly, maybe.

## 1. Introduction

Due to the structural nature of chord progressions, it's feasible that a machine could be taught to emulate human jazz improv. This could be done by stating rules (jazz grammars), but these risk losing the flexibility or fluidity of jazz. Instead of giving an algorithm rules, what if we give it stylistic examples of what we'd like to hear more of? It can determine the features present on its own.

Their approach was **deep belief networks**, or DBNs. These are multi-layer restricted Boltzmann machines (RBMs), which are a type of stochastic (a given input gives a somewhat random output) neural network. They merely focus on the creation of melodies, not at all in a real-time collaborative setting. This is application-based, nothing more.

They believed DBNs might work due to recent work of Hinton, as of 2010. DBNs learn to recognize by creating examples (as bit vectors) and comparing those to training examples. The model then adjusts its parameters to produce more similar examples and is thus unsupervised. This is similar to the way humans emulate behavior. The stochastic part of DBNs helps them to achieve novelty, and thus almost creativity. Their goal is create interesting melodies as opposed to Hinton's of recognizing them.

## 2. Restricted Boltzmann Machines

The **restricted Boltzmann machine** (RBM) was introduced by Smolensky in 1986 and developed by Hinton in 2002-2007. It has two layers of neurons: one visible and one hidden layer. The two layers are fully connected with symmetric, bi-directional weights.

One training cycle for the algorithm takes a binary data vector as input, activating the visible neurons to match the input data. It then alternates activating the hidden nodes based on the visible nodes and vice versa. Each node is activated probabilistically based on a weighted sum of the nodes connected to it. Because nodes within a layer are not connected to other nodes in the layer, the activation of each layer only depends on the other layer. Once the network stabilizes, the new configuration of visible nodes can be read as the output.

The goal of an RBM is to learn the features of sets of data sequences. This paper implemented the *contrastive divergence* (CD) algorithm based on [Hinton's work](http://www.cs.toronto.edu/~fritz/absps/tr00-004.pdf). Their implementation was modeled based on a [tutorial by Radev](http://imonad.com/rbm/restricted-boltzmann-machine/). The CD enabled inexpensive training given the large number of nodes and weight in their network. Once trained, an RBM can take random data sequences and generate new sequences which emulate the features of the training data.

A single RBM can learn some patterns in training data, but multiple RBMs layered together are even more powerful. Such a machine is called a **deep belief network**. The RBMs are combined by identifying each one's hidden layer as the visible layer of the next.  
![DBN Graphic](https://i.imgur.com/TiMcbrv.png)

The second RBM learns features about the features learned by the first, and so on. Thus the DBN can learn far more intricate patterns than a single RBM could alone.

## 3. Data Representation

To train RBNs on musical data, the authors had to encode the music as *bit vectors*, with each beat divided into *slots*. They chose 12 slots per beat to allow both 16th notes and triplets. Each slot ended up as a block of 30 bits, with 12 chord bits and 18 melody bits.

For the melody, they one-hot encoded the twelve note options as twelve slots and then encoded four more slots to specify one of four octaves. One bit indicated a sustained note and one represented rests. When a note is attacked, only its pitch and octave bits were on. If it's being sustained, only the sustain bit is on. Doing their octaves like this saved training time by reducing the number of bits quite a lot.  
![Pitch Encoding](https://i.imgur.com/m7CE4xG.png)

Each chord was one-hot encoded as the 12 possible pitches in the chord. These were simply on-off bits. The melody and chord vectors were concatenated to form part of the input corresponding to one slot. Hopefully, the machine learns to associate specific chords with melodic features.

## 4. Training Data

The network was initially trained on a small set of children's melodies. These were all in the same key and consisted of simple rhythms and notes all in their respective chords. Once they had trained a model to create these melodies, they moved on to larger networks and jazz.

Their main dataset was a large corpus of 4-bar jazz licks (short coherent melodies) cycling over the common $\text{ii}-\text{V}-\text{I}-\text{VI}^7$ turnaround progression in a single key. The $\text{ii}-\text{V}-\text{I}$ cadence is common to jazz, and the $\text{VI}^7$ is the dominant to the next $\text{ii}$ chord. The licks were either transcribed from notable jazz solos or hand constructed, some with help from a "lick generator" from the [Impro-Visor software](https://www.cs.hmc.edu/~keller/jazz/improvisor/). 

## 5. Learning Method

Their goal was to create melodies which transitioned between chords in the progression. To aid this effort, they trained the model on windows of one measure each which cycled forward one beat at a time. These measure-long snapshots continued until the end of the 4-bar lick. In this way, a single training example was broken into 13 overlapping 1-bar windows.

For creating melodies, they started the model with a "seed" of specified chord bits of the desired progression and random melody bits. The chord bits are *clamped* so that they cannot be modified during the creation cycle. In creating the melody, they use a process similar to the windowing used for training. They generate the first few beats of the melody, clamp their bits, and then shift down the melody and chords to make room for the next beat. Thus the machine generates one beat at a time but uses clamped chords and clamped beats of preceding melody.  
![Clamping Process](https://i.imgur.com/k6aZaIG.png)

During the final activation of the network's visible layer (which will be the newly generated melody), they constrain the activation of bits in a specific way. Each slot is looked at individually and only the highest probability of pitch and octave is used. Thus the machine chooses one of sustaining, resting, or starting a new pitch. This approach was found to create a variety of melodies while resonating well with the chords.

They also tested if the machine could learn the progression in arbitrary keys and thus included the option to transpose each input into different keys. All functionality mentioned was implemented in a stand-alone tool they call "[RBM-provisor](https://sourceforge.net/projects/rbm-provisor/)." The tool was written in Java and supports input/output in the leadsheet format from Impro-Visor.

## 6. Results

Their first experiments used children's melodies with 2-layer networks trained for 100 epochs. The results were successful, fitting chords well and flowing melodically. After this success, they moved on to jazz.

For the jazz creation networks, they experimented with the model, varying the number of layers, number of neurons per layer, number of training epochs, and other factors. They settled on a 3-layer network with 1441 input nodes (4 beats with 12 slots each  and 30 bits per slot plus one bias slot) and 750, 375, and 200 hidden nodes respectively. Typical training lasted 250 epochs on about 100 4-bar licks. This took about nine hours on an inexpensive desktop computer.

When analyzing their generated music, they found that most notes were in the chord, with some color tones. Foreign tones were quite rare. Created melodies tended to avoid large jumps and rarely jumped octaves.

![Output Example](https://i.imgur.com/P0ZYODm.png)

The training method was tested with transpositions by training on four copies of the inputs, each transposed up 0, 1, 2, or 3 semitones. The machine was still able to create chord-compatible music regardless of the seed chords. They didn't test with more than 4 transpositions due to training time.

To justify the importance of being able to transpose, jazz chord progressions often have abrupt implied key changes. Thus relative transpositions quickly become important. As an example, it's more economical to train on all transpositions of a $\text{ii}-\text{V}-\text{I}$ than all possible contexts of any one version of the progression.

They noticed some differences between the training data and their generated music:
1. Generated licks tended to avoid half-step intervals. This avoidance of approach notes tended towards chord tones.
2. Rhythmically, the outputs almost entirely consisted of duplet rhythms.

An alternative approach was attempted to select bits using the neuron probability distribution, as opposed to the maximum probability. This produced the disjointed melodies seen above. They also tried encoding strong/weak beat information, but these results were not any better than the chosen encoding.

To be clear, DBNs are not the author's first choice for lick generation. The approach used by Impro-Visor is superior to this one, by far. DBNs also take a while to train. It is possible, though, that DBNs avoid as much algorithmic bias as unsupervised approaches using clustering and Markov chains.

## 7. Future Work

These results are promising, certainly, but further improvements could be made. Their model mainly output duplet rhythms, despite triplets in the training data. This is likely due to an overshadowing of the triplets, but no alternative note generation rule yet found yields as coherent results.

The generated music also included a disproportionate number of repeated notes, which sound a bit static and immobile. They did try post-processing generated music to merge repeated notes, but better solutions would prevent the machine from producing the notes in the first place. Perhaps a different encoding?

They also believe that a similar method could be used to infer chords. Theirs took chords as an input and produced suitable melodies. Another DBN work might be able to determine possible chord progressions for a given melody.

## 8. Related Work

Rossen Radev's tutorial on RBM implementation is again cited. Previous work into music generation has largely already been mentioned in this notebook above.

## 9. Summary

These results show that DBNs can learn to create jazz licks. The approach can work in multiple keys and suggests that, given a significant dataset, similar models might be able to solo over full 12-bar progressions. Their results show novelty but do have minor limitations in rhythm and pitch repetition. The possibility of using DBNs in this way was proven, and further work is likely to be done.

-Fin 1/12/19-

# Source 5: [Composing a melody with LSTM RNNs](https://pdfs.semanticscholar.org/f707/ff253dc44ffa1e15f7ad19d75473a3ddecac.pdf)

This source, unlike the others, is 56 pages in full. As such, I'm going to read through it a bit differently by reading full sections and then looping back to take notes.

## 1. Introduction

This paper's primary goal will be to see to what extent LSTM networks can be used to generate melodies over chord progressions. It will also explain similar prior attempts, go over the algorithms used, explain the implementations for this particular result, and evaluate the generated music by comparing it to music composed by humans using human subjects' judging.

## 2. State of the Art in Algorithmic Composition

There are approaches dating back to 1024AD for algorithmic composition, but the advent of computers enabled many more possibilities. Starting in 1955 with the Illiac Suite and Metastasis, algorithmic composition continued with the likes of Cope's Experiments in Musical Intelligence (1981), Mozer's CONCERT (1994), and the LSTM work of Eck and Schmidhuber in 2002. This thesis was motivated to pick up where Eck left off.

## 3. Neural Networks

### Feedforward Networks

**Feedforward Neural Networks Basics**  
These networks are comprised of neurons, which sum numerous inputs and squash the result using some activation function. For any one neuron $k$, each input $x_i, i=1,2,...,m$ is multiplied by its own weight $w_{ki}$. The net input $v_k$ is the weighted sum over all inputs.

There's also a single bias $b_k$ feeding into the neuron. This bias can be set to weight $w_{k0}$, where $x_0=1$. We can then set $v_k=\sum_{i=0}^mw_{ki}x_i$.

The final output, or activation $y_k$, of the neuron applies some activation function $\varphi(\cdot)$ to net input $v_k$:  
$y_k=\varphi(v_k)$.

Of the many activation functions used, the $\text{sigmoid}$ function is quite common. It squashes outputs of the neuron between 0 and 1:  
$\sigma(v_k)=\frac{1}{1+e^{-a*v_k}}$, where slope parameter $a$ can vary as needed.

**Network Architecture**  
A given number of these neurons are in each row of a neural network. Each row is fully connected to the next by *synapses*, which each have the weights we're familiar with. The first layer in a network is called the *input layer* and the last is the *output layer*. In networks with more than two layers, middle layers are called *hidden layers*.  
![Multi-layer Network](https://i.imgur.com/PgIZUrL.png)

**Learning through Backpropagation**  
These networks need some way to learn, to change their synaptic weights to output closer to the desired results. They'll learn from training data ${(x^{(1)},t^{(1)}),(x^{(2)},t^{(2)}),...,(x^{(n)},t^{(n)})}$, with inputs $x^{(n)}$ and expected targets $t^{(n)}$, where $n$ is the number of examples.

We need some *loss function* to evaluate the difference between the target values $t_j$ and the actual network outputs $y_j$. The squared error function is often used for this, where $j$ refers to the $j$-th neuron of the $J$ in the output layer.  
$E(w_{ki})=\frac{1}{2}\sum_{j\in{J}}(y_j-t_j)^2$.

Once we have this metric to see how far off we are, we still need to update our weights to improve the network's output. This goal is synonymous with minimizing the loss, and thus we can use the *gradient descent* method introduced [in 1986](https://www.iro.umontreal.ca/~vincentp/ift3395/lectures/backprop_old.pdf). The general update rule of this method follows:  
$w_{ki}:=w_{ki}-\alpha\frac{\delta}{\delta(w_{ki})}E(w_{ki})$, with learning rate $\alpha$.

**Computing Partial Derivatives**  
This formula calls for the calculation of the partial of $E(w_{ki})$ w.r.t. weights $w_{ki}$. This is done in two steps: one for the output layer and one for the hidden layers.

**Weights in Output Layer**  
By the definition of $E$, we have $\frac{\delta{E}}{\delta{w_{ji}}}=\frac{\delta}{\delta{w_{ji}}}\frac{1}{2}\sum_{j\in{J}}(y_j-t_j)^2$.  
$=> \frac{\delta{E}}{\delta{w_{ji}}}=(y_j-t_j)\frac{\delta}{\delta{w_{ji}}}y_j$. This is because the partial of $E$ w.r.t. one $w_{ji}$ will only involve that specific $j$; all other terms of the sum will be zero. The chain rule explains the rest, as $t_j$ is constant.

Recall that we had $y_k=\varphi(v_k)$.  
Then $\frac{\delta{E}}{\delta{w_{ji}}}=(y_j-t_j)*\frac{\delta}{\delta{w_{ji}}}\varphi(v_j)$. Chain rule again to get:  
$\frac{\delta{E}}{\delta{w_{ji}}}=(y_j-t_j)*\varphi'(v_j)*\frac{\delta}{\delta{w_{ji}}}v_j$.

Because $v_k=\sum_{i=0}^mw_{ki}x_i$, we have $\frac{\delta}{\delta{w_{ji}}}v_j=x_i$ by all other terms equalling zero.  
$=> \frac{\delta{E}}{\delta{w_{ji}}}=(y_j-t_j)*\varphi'(v_j)*x_i$.

For the sake of simplicity we also define $\delta_j:=(y_j-t_j)*\varphi'(v_j)$. Finally:  
$\frac{\delta{E}}{\delta{w_{ji}}}=\delta_jx_i$.

**Weights between Hidden Layers**  
We now consider weights $w_{ki}^{(l)}$, where such a weight connects the $i$-th neuron in layer $l-1$ to the $k$-th neuron in hidden layer $l$. We unfortunately cannot omit the sum this time, as each $y_j$ depends on all previous weights in the network.

We again start with $\frac{\delta{E}}{\delta{w_{ki}}}=\frac{\delta}{\delta{w_{ki}}}\frac{1}{2}\sum_{j\in{J}}(y_j-t_j)^2$.  
$=> \frac{\delta{E}}{\delta{w_{ki}}}=\sum_{j\in{J}}(y_j-t_j)\frac{\delta}{\delta{w_{ki}}}y_j$

Again, with $y_j=\varphi(v_j)$ we have $\frac{\delta{E}}{\delta{w_{ki}}}=\sum_{j\in{J}}(y_j-t_j)\frac{\delta}{\delta{w_{ki}}}\varphi(v_j)$.  
$=> \frac{\delta{E}}{\delta{w_{ki}}}=\sum_{j\in{J}}(y_j-t_j)\varphi'(v_j)\frac{\delta}{\delta{w_{ki}}}v_j$.

I'm going to skip over this section for now. It's intense calculus and this is clearly worth fully studying on my own at some point. For now, I'd like to cover LSTMs to stay on-topic.

### RNNs

RNNs can capture patterns over time, which is crucial to music production. Each neuron's output is fed back into it in each consecutive time step.

**Backpropagation Through Time**  
Unfolding an RNN through time will result in a feedforward network. One can then apply a modified version of backpropagation to update the weights of the RNN.

Unfortunately, music composed with RNNs was found to be "not musically coherent" by Moser in 1994. This is due to the limited memory duration of the model, which is caused by a tendency towards exploding or vanishing gradients. LSTMs were introduced to solve this problem.

**LSTM RNNs**  
LSTMs were designed to avoid the "rapid decay of backpropagated error" found in typical RNNs. This model uses a "memory block" (cell state) to store and recall memory. It has three gates which act as read, write, and reset functions. These gates are better discussed in my other notes specifically on the topic.

LSTMs can be trained with BPTT as well, although they need numeric data. Thus we'll need to represent music in this way, somehow.

## 4. Data Representation: MIDI

In choosing a representation for music to feed into the LSTM, we have the options of raw audio or MIDI. What are the benefits to each, you may ask? Let's check:

Our goal is to compose melodies for given chords, only caring about which notes ae held for what duration of time. We thus disregard velocity, articulation, or dynamics.

**Audio** - It's a rather complex undertaking to reduce audio signals to their mere pitches. We'd need to Fourier Transform to get base frequencies, map frequencies, line up transients, and map timesteps.

**MIDI** - MIDI is the standardized data format for musical control data in digital instruments. MIDI only includes the start, duration, and end of pitches. It thus provides all we need for LSTMs and only needs to be converted into an appropriate numeric representation. Thus the author chose to use MIDI.

### Piano Roll Representation

This thesis used a representation similar to a piano roll, with a 2D matrix of 0s and 1s. MIDI ticks are 96 per beat and thus the data was upscaled to 4 ticks per beat. The vertical axis of the matrix is time and the horizontal the pitches. The range of the pitches included was only the notes in the training set. Overall the matrix was size $(\text{num of }\frac{1}{16} \text{ steps},\text{note range})$.

Notes were indicated as "on" by a value of 1 and "off" by 0 for any one time step. This representation has the issue that multiple of the same note consecutively is identical to one longer sustained note.

The author then adjusts their representation so that the second half of each note is denoted by 0. This leads to a need for two time steps for each note. For 4 ticks in a beat, this would only allow a resolution of eighth notes. Thus the final resolution is raised to 8 ticks per beat, which still allows that initial resolution of 16th notes.

## 5. Implementation

The author chose to use Keras in Python. Mido was used to deal with the MIDI data. Overall, the implementation had two programs:
1. Training the LSTM.
2. Composing a melody for a new chord sequence.  
![The Two Programs](https://i.imgur.com/RKzTqaW.png)

### The Training Program

**Network Inputs**  
The training dataset consists of chord sequence inputs and the belonging melodies as targets. The MIDI has 96 ticks per beat, and all times in the MIDI encoding are relative to each other; there is no absolute time. In this way the melody and chords are turned into the matrices.

For Keras, the inputs need further transformation. During training, several timesteps of chords will form a 2D matrix. The network will then output a vector predicting which notes to play next.

The number of timesteps in the chord sequence is denoted by $n$. The target vector is then the $n+1$ timestep from the melody piano roll.

The Keras framework LSTM requires 3D input matrices of size $(\text{# samples}, \text{timesteps}, \text{input dimension})$ and 2D target matrices of size $(\text{# samples}, \text{output dimension})$. Here, $\text{timesteps}$ refers to the sequence length $n$. The $\text{input dimension}$ is the number of pitches possible for the chords. The $\text{output dimension}$ is similar for the melody's pitch range. The number of samples is the difference between the number of total timesteps in the song and the sequence length.

The Network Input Matrices are created by taking a sample of size $(n, \text{chord pitch range})$ from the start of the chord piano roll. The following samples are then created by scrolling the window of length $n$ along the matrix timestep by timestep. This goes until the window includes the penultimate timestep of the matrix. The Prediction Target Matrices take the melody piano roll at each timestep right after the Network Input Matrices.

Personal notes: this encoding used 8 timesteps per beat, and mine needs 24 with the current time scale I'm allowing if I use this way to end notes. I'd need $n=24$ to replicate his scale of LSTM inputs, so maybe I should consider other encoding methods.

**Network Properties**  
An LSTM RNN consists of an input layer, output layer, and optional hidden layers. The input layer includes input nodes fully connected to the next layer. This architecture used 12 input nodes and 24 output nodes. This gives an octave for chords and two octaves for melodies. The program leaves it up to the user to vary up the number of hidden layers and their contents.  
![Possible LSTM Structure](https://i.imgur.com/z0W6Smb.png)

### The Composition Program

After the LSTM is trained, it can be used to compose new melodies for given chord sequences. This process starts with a given MIDI chord progression compressed into one octave. We can then piano roll this MIDI, turn it into a Network Input Matrix, and feed this into the LSTM.

The model will predict output values for each timestep example of the input. The resulting Prediction Matrix will consist of values between 0 and 1. At each timestep, the highest value is found. If this value is higher than a given value, it's set to 1 and otherwise all values in the timestep are set to 0. We thus get a monophonic melody piano roll.

We then need to create MIDI from the piano roll. This is done in a reverse of the process used to create the piano rolls initially. This concludes the composition program.

## 6. Experiments

In this section we'll describe the training and testing data and then explain the experimental training process.

### Training/Test Datasets

**Training Data**  
The decision for what to use as training data is a crucial one, for this in itself defines the compositional goal of the network. This thesis had two goals for LSTMs:  
1. Compose music indiscernible from human-composed music.
2. The melodies should simply sound pleasant.

This second metric is difficult to measure, so the author chose to use music appealing to a broad demographic: The Beatles. The author's justification was pretty obvious: their success and popularity. The results also needed to be short in duration, as test subjects would be listening to multiple examples. This was set to 8 bars.

The training data was taken from 16 different Beatles songs from the book "Pop Classics For Piano: The Very Best Of The Beatles - Easy Arrangements for Piano." 68 8-bar samples were taken from these songs. The chords' notes ranged from C2 to B2, and the melodies from C3 and B4. Quantization was to the 16th note.

**Test Data**  
This was a set of previously unseen chord sequences which the LSTM would compose melodies over. 4 of the 8 test progressions included chords which weren't in the training dataset, while the other 4 were comprised of the most common chords.  
![Test Chord Progressions](https://i.imgur.com/5UmiNnZ.png)

### Training Different LSTM Topologies

There no existing heuristics to decide the optimal topology for LSTMs and music composition. The author therefore chose the best topology out of eleven based on its compositions on training data.

The networks tested included zero to four hidden layers. The number of LSTM blocks per layer was chosen as multiples or fractions of the number of input nodes (12). The experiment started with 0 hidden layers, and added hidden layers as it went. As hidden layers were added, the melodies were more and more cohesive up until networks with 4 layers were more monotonous. Thus training stopped after 4 hidden layers.

![Topologies Tested](https://i.imgur.com/QbSSpf8.png)

### Network Compositions

Each network composed melodies to the eight test chord sequences. The LSTMs with more hidden layers played fewer notes, in general, until the 4-layer LSTM was often stuck on a single note or not playing at all. The LSTMs in general learned to play diatonically to the key.

One issue was that the networks often played notes off the beat, either an 8th or 16th after what might be expected. Thus the beginning and ending of all notes were adjusted to the nearest 8th note.

Of the 11 topologies, 3 were pre-selected and tested in subjective listening tests with 3 participants. The seventh network topology's melodies were selected as the most pleasant and human-sounding. These melodies were used in a listening test.

## 7. Evaluation

The melodies which the seventh LSTM produced were tested in an evaluation similar to the framework presented by [Pearce](http://webprojects.eecs.qmul.ac.uk/marcusp/papers/PearceEtAl2001.pdf). The two compositional goals were tested with human listeners. The test was developed to be taken online and 109 people participated. The test was broken into two parts:

**Part 1**  
The author wanted to determine if humans prefered LSTM compositions or human ones. Four "musically skilled persons" were each asked to compose melodies over two of the eight test chord progressions. Each pair of LSTM-human melodies over the same chord progression was presented as a pair. Participants were asked which melody they liked more.


**Part 2**  
This section aimed to determine if the LSTM compositions are recognizably not by humans. At this point, participants were told some melodies were written by a computer. Given one of the sixteen melodies, they were asked if each was written by a human or computer. At the end they were also asked for reasons for their decisions.

Participants were also asked about themselves to get general imformation about their musical background. The factors asked about were based on [this paper from 2014](https://journals.plos.org/plosone/article?id=10.1371/journal.pone.0089642).

**Results**  
The participants prefered human compositions to computer-generated ones, although the computer melodies could not be identified as such on their own. The demographic was largely quite musical (64.22% played instrument for >3 years). The LSTM melodies were prefered about 26% of the time on chord sequences 1-4, and about 29% on sequences 5-8. Maybe the slight gap is due to forced creativity?

Overall, the LSTM managed to create prefered melodies in 27% of cases. That's not bad. The computer melodies were correctly identified about 60 of the time, which is again not bad.

## 8. Conclusion

The LSTM managed to somewhat meet the goals for the thesis, although improvements could be made. Possible future improvements are suggested:
* More training data
* More acute compositional goals and tailored data for these
* Another representation style

*-Fin 1/15/19-*

# Source 6: [Music Generation from MIDI datasets](https://neuro.cs.ut.ee/wp-content/uploads/2018/02/MIDI_music.pdf)

**Abstract** - This work employed a character-based RNN to generate music. In previous works, RNNs have succeeded in modeling sequential data.

## 1. Introduction

This project's aim was to replicate some previous approaches in using RNNs. Prior to this paper, RNNs as well as CBOW, sequence-to-sequence, or GANs have been tried for music generation. This paper used MIDI data quantized to specific note lengths, and then specified one character for each note when it's played.

##2. Background

Machine improvisation is a relatively new concept which uses AI and ML to capture the characteristics of music. These models use available examples and glean insight from them. Especially of note are neural models, which are layered computational graphs. In each layer of these graphs, more sophisticated higher level features are represented. For music generation, LSTM cells are even more powerful.

The paper by N. Agarwala generated monophonic and polyphonic music using multiple methods. Their RNN reached 60% accuracy and sequence-to-sequence reached 65% accuracy. The latter fooled 75% of humans if the compositions were by human or machine. They also experimented with GANs, but no useful results were found.

Agarwala found that char-RNNs benefitted from increased hidden layer size and shorter embedding size, while Seq2Seq benefitted from increased hidden layer and embedding size. They used ABC notation, as MIDI requires less of a learning system. Thus ABC creates models able to learn the grammar of music.

Bob L. Sturm used character- and token-based approaches with LSTMs. Sturm's work also created a statistical analysis technique which helps relate LSTM compositions to originals. This gives a recipe for regularizing the RNN models to generate grammatical music.

## 3. Methods

**Data Preparation**  
They used textual representations of music extracted from MIDI files. MIDI files themselves are collections of tracks, each with events such as notes on/off, tempo, instrument-type, and meta data. The tick number of each MIDI event is relative, describing how many ticks passed since the previous event. They define the resolution as the pulses per quarter-note (PPQ), which is the number of timesteps per quarter note. In their formatting, they treated events from separate tracks as events on one track.

They quantized each MIDI event by a specific divider of the MIDI resolution, starting with 8 and then choosing 4 in the end. These 16th-note time steps are used to extract all notes in the MIDI data. Each note pitch is assigned its own ASCII character, and these characters are concatenated together to represent chords for each time step. Each time step is then separated by a space.

In their tests, they found that the length of generated MIDI music depends on the quantization length as well as the note types in question. They thus needed to normalize their data so that all pieces were similar in tempo. The tick timing of the MIDI events were morphed to the same tempo (I emailed the author asking what exactly this means). Somehow the tempos were doubled or halved until they matched up? Awaiting that email back.

The dataset was also augmented by transposing the data up/down one or more semitones, which might help the model learn a more varied harmony.

**Network Architecture**  
The author trained a char-RNN with LSTM cells in a many-to-one architecture. They used sentences from the dataset as input and the character following this sentence was used as a label. Input substrings were equal in length to the number of RNN time steps, and this sliding window was moved over all the dataset texts. The inputs AND outputs were all one-hot vectors of the specific vocabulary of note characters.  
![Char-RNN Architecture](https://i.imgur.com/lXjaoI0.png)

A dense layer with softmax activation was used to spit out a one-hot vector of probabilities. Each value describes the probability of each possible character. The dataset was also split into training and validation sets by randomly choosing tracks until a desired percentage (of the total length of the text data) was in the test set.

**Sampling**  
Afte training, new music could be generated as text. The network was fed a warmup sequence (either a random sequence, a sentence from the datatset, or another sentence in the vocabulary) and the next character was predicted. These new predictions are appended to the sentence and the input slides over one character.

As mentioned, the output is a vector of character probabilities. Before one is chosen, the values are divided by some temperature value. A higher temperature leads to more uniform distributions. Apparently, this concept is similar to the simulated annealing algorithm.

Source 7 is secretly [here](https://cs.stackexchange.com/questions/79241/what-is-temperature-in-lstm-and-neural-networks-generally), which basically explained how temperature is a parameter for LSTMs which forces one of two things: a high temperature will tend all outcomes towards the same probability, while a low temperature near 0 would discourage too much exploration and shift the most likely candidate towards a probability of 1.

**Determine over-fitting**  
The authors compared wave signals of the training dataset and the generated music to check for correlations. This was done with Fast Fourier Transform, autocorrelation, and then by measuring the maximum correlation points between two pieces. This correlation was performed in the frequency domain and thus is versatile because it does not depend on sequences. They used this to avoid textual pattern analysis, although this method might be helpful for my work.

## 4. Experiments and Results

They implemented their model using Keras and trained it on the following data:
* 194 Bach pieces, 1.4M characters
* 39 Mozart pieces, 0.4M characters
* 335 tracks by multiple composers, pianomidide, filtered down to 132 tracks with 0.8M characters

They also varied the structure of their network and the data formatting during experimentation. What helped was to use accuracy evaluation with training/validation sets, tempo normalization, data augmentation, and batch data shuffling for each epoch. The network's parameters included categorical cross entropy loss, Adam optimizer, a 0.001 learning rate, and a validation split of 0.2.

The best musical result used a many-to-one RNN with one LSTM layer of 512 units, sentence length of 100, which was trained on the Mozart data with normalized tempos and all data transposed by 0 to 5 semitones.  
![LSTMs Tested](https://i.imgur.com/pwTfqZ1.png)

They never were able to train using the bigger Bach dataset with augmentation, as they ran into memory issues. The following parameters were the most notable:
* Sentence length (RNN time steps) - With quantization of 4, 100 as the sentence length worked well enough.
* Many-to-many architecture (outputting next $n$ characters) didn't improve much of anything.
* The number of network parameters (which depends on the number of LSTM layers/units) should roughly match the size (# of chars) of the dataset in magnitude. Their logic is that it might be beneficial to slightly overfit to reproduce harmonic combinations.
* Tempo normalization - This alleviated an initial problem they encountered, where the network would repeatedly produce one chord for long stretches.
* Transposing notes to different keys was a great way to improve the harmonies of the generated tracks.

Their model was not simply reproducing the training data, as there was little correlation between training sets and generated pieces.

## 5. Discussion

The implemented approach gave decent results, but it was bound the limiting knowledge which the input format could provide. Experiments with the temperature parameter demonstrated that the network has not seen enough data, as the lower temperatures caused repeated notes/chords in the generated music.

They note that the temperature parameter gives the benefit of randomness, which means that the sampling is very unlikely to completely imitate the training data. Further analysis of the effects of this parameter are needed.

## 6. Future Works

Here's their ideas for continuing on this field:
* Other RNN architectures
* Encoder-decoder architecture of Seq2Seq model
* Different augmentations to different keys to improve results' harmonies
* Fully training using all datasets with large data augmentation
* Training on different datasets or genres
* Other music notations
* Incorporating MIDI velocity (as in volume of notes)
* Multiple tracks or instruments
* Textual pattern matching to determine overfitting more

## 7. Conclusion

This paper implemented a char-based RNN with LSTM cells on classic piano pieces obtained as MIDI files. The best model had training accuracy above 90% and about 70% validation accuracy. The model initially generated repeated chords or notes, but normalizing the tempos and augmenting the data eliminated this issue. They also used autocorrelation to measure overfitting. Their results can be sampled [here](https://yellow-ray.de/~moritz/midi_rnn/examples.html).  
*-Notes finished 1/18/2019-*

# Source 7: [Music Composition using Recurrent Neural Networks](https://pdfs.semanticscholar.org/c933/79a401dd159fc0c90eab44c43d07286b227e.pdf)

**Abstract**  
This work proposes generative RNN models to create sheet music with well-formed structure without predefining music composition rules to the models. Char-RNNs are able to learn some patterns but not structurally accurate music, fooling only 35% of humans. GANs were tried, but produced no meaningful results due to unstable training. Seq2Seq models did quite well, producing pleasing and structurally coherent music with a test accuracy of 65% and fooling 70% of human listeners.

## 1. Introduction

A key note is that music has to be both expressive and precise. Is it possible to model both using deep learning? RNNs have shown success with text generation, question-answering, chat bots, and sentiment analysis. This work builds RNNs to generate music using char-based representations, using char-RNNs, Seq2Seq, and GRU and LSTM cells with bi-directionality and attention (???). Their music was represented in ABC, as this simulates the environment in which humans create music. MIDI and MFCCs have limitations in their ability to create sheet music.

## 2. Related Work

Most previous work in music composition has used MIDI, as I've well seen. In cases where raw audio was used, mel-frequency cepstrum coefficients (MFCCs) were used to approximate the human perception of sound by weighting pitches as the human ear perceives them. Both data forms are continuous, which is why converting them to sheet music may be inaccurate or even impossible.

Sheet music is inherently discrete. To overcome this, previous work with MIDI has used assumptions to quantize the notes into 4/4 and/or the key of C. Notes faster than 16ths are also ignored in this way. This simplification implies that the models need not *compose* music, as the outputs will *always* be able to be mapped back to valid sheet music. Thus these models learn only the combination of sounds, not musical syntax.

The alternative to the issues with MIDI is to use the ABC notation, which defines a mapping between characters and specific symbols in sheet music. This forces a model to learn the grammar of music (like barlines), which they argue is superior and necessary.

A few previous models used RNNs with RBMs or char-RNNs. The former created polyphonic music with piano rolls and the latter Irish folk music. This work focuses on monophonic composition across different genres.

## 3. Dataset and Preprocessing

**Data Acquisition**  
There's not really a data repository for ABC formatted music yet. The authors had to collect data from ABC sites abcnotation.com and thesession.com. In total, these provided 17,500 and 16,500 songs respectively. The songs comprised 19 genres.

**ABC Music Notation and Their Encoding**  
ABC notation has two parts: the header, which includes song metadata, and the body, which details the notes in the song. Spaces in the body indicate new notes purely for legibility, but they're unneeded for music generation. The scraped songs were processed to remove this whitespace. Only song type, time signature, unit note length, number of flats, and song mode were kept from the headings. Two more metadata categories were created: # of measures and song complexity (average # of notes per beat).  
![Example ABC Formatting](https://i.imgur.com/0oVYqWR.png)

The dataset was also augmented by transposing all songs to 4 random keys, which involved changing the pitch of each song by some constant interval. Transposed songs are valid songs because the note relationships remain intact. After formatting the ABC files, the characters were mapped to unique integer encodings and a keyword to each category of the metadata. The encoded data was split 80/10/10 into train/test/dev sets.

## 4. Methodology

### Continuous Bag-of-Words

They used this as a trivial baseline. CBOW typically uses the context window around a word to predict the context word, using a simple feedforward network. This application called for predicting the next musical character given a history, so the context window was the previous $n$ (a hyperparameter) characters. CBOW averages the character embeddings of the one-hot encoded context window in the hidden layer and then predicts probabilities.

### Char-RNN

Traditional neural-language models use word embeddings, which require them to predict probabilities over more than 10,000 words. Thus using character embeddings creates an easier model to train. Encoding the notes directly defeats the purpose of the project, which is the specific learning of musical structure *and syntax*.

**Char-RNN** - This simple RNN model has $n$ layers of recurrent units each unrolled into $T$ time steps. These $T$ time steps share the same weights and have hidden size $h$. One hot vectors from formatted ABC data are converted into character embeddings using a shared weight matrix and then passed through the char-RNN. These outputs are passed through a shared weights matrix and a softmax to get the probabilities for the next character.

The initial hidden state was not set to a zero-vector. Instead, they created a shared weight matrix for the metadata encodings and concatenated all the embeddings into the initial hidden state. (I have no idea what this means right now)

### Seq2Seq

Seq2Seq models were created to perform better than LSTMs on predictions based on long sequences. They have an encode and decode stage.  
Encode stage - Normal char-RNN without output weights or softmax. A character sequence $L_1$ is used to create a thought vector $v$, the final hidden state after unrolling $L_1$ time steps.  
Decode stage - Another Char-RNN takes $v$ as an initial state prepended with a $<go>$ token. The targets of the decoder are the next $L_2$ characters in the input.

### Generative Adversarial Networks

GANs didn't work in their experiments. I'm going to ignore this section for this project.

## 5. Results

### CBOW

No hyperparameter tuning was conducted, so $n$ was set to 15. The model achieved about 20% accuracy on the training and testing sets.

### Char-RNN

They assumed that input size and the rest of the hyperparameters were independent due to time constraints. They thus had two steps:
1. Grid search key hyperparameters.
2. Input size comparison on optimized LSTM.

**Grid Search**  
They used early stopping at 2 epochs with 3 parameters (hidden state size, music body embedding size, and dropout ratio). The 18 configurations were done with RNN, GRU, and LSTM cells. After 2 epochs, RNN had 39.5%, GRU 47.5%, and LSTM 51.7% on the dev set. For the rest of the project, only LSTM cells were considered.

They then did a more granular search over 72 configurations. In general, larger hidden size and smaller embedding seemed to help. They did not make the hidden size larger than 1000, as that tanked the training speed. Thus a hidden size of 800 was chosen, dropout 0.2, and embedding of size 20 were chosen. The model also improved with larger input windows, with 60% on windows of size 50. The final result was 59.5% on the test set.

It was difficult to judge their results, so they tested their model against Karpathy, who claimed 58% prediction on *War and Peace*. Their model, with window size 25 and the best hyperparameter setting from before, was able to achieve 60%.

### Seq2Seq

Seq2Seq is an even slower model, so early stopping was set to 1 epoch and the input size was set to 25. They tuned hidden state size, embedding size, attention mechanism type, and the directionality of cells. Larger hidden state and larger embedding size increased accuracy. Accuracy reached its maximum at around 100 embedding size.

The Bahdanau attention model was a bit better than the Luong, and directionality of cells had little impact on accuracy. This entire paragraph is stuff that I don't understand.

In the end, they chose a hidden state of 800, embedding size 100, unidirectional cells, and Bahdanau attention. This achieved 65.5% on the test set.  
![Seq2Seq Tuning](https://i.imgur.com/mumTv8B.png)

### GAN

The only note is that they remark "Maybe a Seq2Seq model could learn to be a better generator than the things we tried." Otherwise no good results.

## 6. Discussion

After each model was trained, it was used to generate music samples. The models were warm started by songs composed by humans.

**CBOW** was unable to generate anything musical. Its outputs were so bad they couldn't be converted to sheet music, and thus it's concluded that CBOW does not learn anything about musical structure.

**Char-RNN** generated samples which could be converted into sheet music. Thus it learned the character-to-note mapping on its own to output reasonable frequencies of each character. This is supported by the confusion matrix and PCA of the embedding matrix, which show that the largest mislabelings were within the note characters. Thus the model understood that note characters have similar functionalities. The PCA also shows an "octave vector" mapping upper and lower case characters.  
![Confusion Matrix and PCA](https://i.imgur.com/vXM875c.png)

The output's largest drawback was that it failed to space barlines consistently. Thus its output fails to be syntactically correct.  
![Example char-RNN Result](https://i.imgur.com/1badLN4.png)

**Seq2Seq** was able to learn both the functionality of notes and the placement of barlines. The outputs are syntactically correct and even at times nuanced: in an example the model used warm start notes as a motif in its melody.

### Pushing Seq2Seq Further

These promising results prompted further experiments with Seq2Seq models. They converted songs from more genres into ABC format and used these as warm start notes. The model was still able to create a motif of the warm start sequence.

As an additional challenge, the model was trained on duets. In ABC format, these are tricky due to alternating lines, so the Seq2Seq needs to learn to correspond the beats in each voice. This new dataset was 1300 songs and was so small due to the few ABC duets available. The model had minor mistakes, like putting repeats in different places between the two voices, but the overall syntax was correct.

### Effects of Metadata

The best Seq2Seq model was trained with a zero-initialized hidden vector. The overall convergence was the same, but the meta data did provide a slight learning head start.

### Survey

There's no standard and concrete metric to measure the quality of music, so the authors conducted a survey to receive objective feedback on their generated songs. Participants were asked were asked to judge if samples were human composed or computer generated. Of the 3 songs judged to be human composed more than 50% of the time, 2 were generated by the seq2seq model.

## 7. Conclusion and Future Work

Their models were able to learn to compose music that's both syntactically correct and able to fool humans. The flexibility of their best model (Seq2Seq) was able to explore other areas of composition, such as duet songs.

**Future Goals**  
They'd like to fix the GAN model to create something even more musical. They also may extend the project with other music notations, such as MusicXML. Their code can be found [here](https://github.com/yinoue93/CS224N_proj).

# Source 8: [JamBot: Music Theory Aware Chord Based Generation of Polyphonic Music with LSTMs](https://www.tik.ee.ethz.ch/file/7c6a36714f7b64306d2f5b06e14da5c0/JamBot_CameraReady.pdf)

**Abstract** - They propose a novel approach for generating polyphonic music based on LSTMs. There are two steps:
1. A chord LSTM predicts a chord progression based on a chord embedding.
2. A second LSTM generates polyphonic music from the predicted chord progression.  

This results in pleasing, harmonic music with long-term structure. They evaluate the learned chord embeddings to show that the model is sensible from a music theory perspective.

## 1. Introduction

Recent neural networks have increasingly ventured into creative endeavors. This work introduces JamBot, a music-theory-aware system. Earlier approaches included uses of LSTMs, and some even included considerations of chords. In such cases, other models saw chords and melody as separate entities.

JamBot does not separate chords and melody. There's one chord for every 8 time steps, and thus separate LSTMs are needed. The chord structure is fed into a polyphonic LSTM that outputs the actual music. This latter LSTM is free to predict any note; the chords are only provided as information and not as a rule.

The model also learned a chord embedding the corresponds quite well to the circle of fifths. Thus the LSTM is capable of extracting important music theory concepts from data.

**Related Works** - Mozer used an RNN with music theory encoded into the data representation. Eck and Schmidhuber used an LSTM to repeat a blues chord progression. Boulanger-Lewandowski proposed a model that predicted polyphonic music. Other works included Hadjeres, which created 4-part Bach chorales, and Johnson, which also had some long term structure.

Some approaches consider chord progressions, like Choi's. Theirs learned relationships within text documents representing chord progressions. Chu et al. generated a monophonic melody and then chords, drums. Huang and Wu experimented with learned embeddings, Oord created Wavenet, which can generate music.

The authors note that systems using MIDI files produce better sounding music than audio-based approaches. Finally, Avia and Jukedeck join the field as startups instead of members of the scientific community.

## 2. Basics of Music Theory

This section introduces some basic music theory principles used in this paper, including:
* Bar - Specific # of beats, mostly 4/4.
* Equal temperament - All notes are the same ratio apart.
* Scale - Subset of 7 notes over an octave. First note is root note, a root note and a scale give us a key.
* Chords - Set of multiple notes. Major is uppercase of the root, minor adds an m.
* Circle of fifths - The relationship between the 12 notes and their keys.

## 3. Dataset

**General Description**  
They trained their models using a subset of the Lakh MIDI Dataset. Overall, this has 100,000 MIDI songs. As MIDI contains no actual sound data, it's more space-efficient than audio. MIDI's therefore easier to have more of for a model.

**Preprocessing**  
They analyzed the scales and keys of their data, between major, natural or harmonic or melodic minor, and blues. As major/natural minor have the same notes, there are $12*4$ possible keys. The keys were calculated using histograms of the top 7 notes in the songs.

Of the 114,988 songs in the dataset, 86,711 were major/relative minor, 1,600 harmonic minor, 765 blues, and 654 in melodic minor. The other 25,258 were in another scale or had an undetected modulation.

They used only songs in the major/minor scales, and shifted all songs to C. This step was taken only to avoid overfitting due to a lack of data for each key on its own. Post-generation songs can be transposed later on. The notes in this resulting data set were as follows:  
![Note Histogram](https://i.imgur.com/OUZrqPL.png)

MIDI allows for a range of 128 pitches, $\text{C-1}-\text{G9}$. The general tendency is of course towards mid-range notes, and the LSTM thus lacks sufficient data on the far-out notes. Becuase of this, they consider notes from their data only in the range $\text{C2}-\text{C6}$.

To train the LSTM, they needed to extract chords from the songs. They automated this process by computing histograms of the twelve notes for each bar. The top three notes made up the chord. This approximation has its flaws, but its results are feasible and thus they justify its use:  
![Chords](https://i.imgur.com/SlltyXk.png)

## 4. Models

### Description of LSTM

Of course dependencies are important in songs. Context provides alternative meanings to all temporal data points, but classic neural networks (MLPs) cannot handle this challenge. RNNs were developed to solve this, and LSTMs were then made to handle long-term dependencies such as those in music.

The LSTM model introduces a gating architecture into the RNN model. These gates control forgetting, input into, and outputting the "hidden state" of the LSTM. The four gates (update, input, forget, and output) have the following outputs:  
![Gate Formulas](https://i.imgur.com/X6VIEXY.png)


All the $U$s and $V$s are weight matrices and bias terms were omitted for clarity. The memory cell state is updated as a function of the input and previous state:  
$c_t=f\odot c_{t-1}+i\odot g$.

The hidden state is a function of the cell state and the output gate, and the output is the weighted activation of the hidden state:  
$h_t=o\odot \text{tanh}(c_t)$.  
$y_t=\delta(W_{out}h_t)$.

### Data Representation

**Polyphonic LSTM**  
This LSTM's input data was in a piano roll format with a quantization of eight time steps per measure. The notes at each time step are vectors of the length of possible notes. (one-hot basically) These were made with the pretty_midi library.

**Chord LSTM**  
They borrow an approach from NLP where certain words are replaced by integer ids. This limits vocabulary size so that only the $N$ most occuring words receive a unique id and the rest are given an unknown tag.

The chords receive the same treatment, being replaced by ids where the chord/id pairs are stored in a dictionary. There are 1,465 possible note combinations for the chords, but only 300 appeared in the dataset. They chose a vocabulary size of 50 and the rest of the chords were given the unknown tag.  
![Chord Occurances](https://i.imgur.com/2osDHkr.png)

These chord tags were encoded as vectors using one-hot encoding. The input vectors were the same size as the chord vocabulary and were all-0 except a single 1 for the specific chord applicable.

### Chord LSTM

This LSTM's first layer also used word embeddings. This technique, initially from NLP, maps words from the vocabulary as vectors of real numbers. These embeddings are learned from the training data and in the end hopefully capture relationships between similar words.

The one-hot vectors described are multiplied by an embedding matrix $W_{embed}$ to get a 10-dimensional embedded chord vector. The goal here is that the LSTM learns meaningful representations of the chords. The parameters of $W_{embed}$ are learnable and trained with the rest of the LSTM.

Following this layer, the embedded chords are fed into an LSTM with 256 hidden cells. Output activation was $\text{softmax}$. The output of the LSTM corresponds to a vector with probabilities for the next chord.

The chord LSTM is trained using cross-entropy as a loss function and the Adam optimizer. The best initial learning rate was found to be $10^{-5}$. The training data was extracted chords from 80,000 songs in the shifted dataset. Training lasted 4 epochs. A second chord LSTM was trained with 100,000 songs from the unshifted dataset to visualize the learned embeddings.

To generate new chord progressions, a seed of variable length is fed into the LSTM. They sample the output probabilities (with temperature) for the next chords. The next chord is fed into the LSTM and the process repeats. The temperature parameter controls the diversity of the generated chords, where $t=0$ causes an identical output each time.

### Polyphonic LSTM

The input for this LSTM consists of vectors from the song piano rolls with additional appended features. These features are:
1. Embedded chord of the next time step. (Same as that in the trained chord LSTM)
2. Embedded vector of the chord that follows the one of the next time step. This encourages the melody to anticipate these notes.
3. A binary counter that counts from 0 to 7 each bar. This enables the LSTM to understand where it is in time.

**Architecture**  
Input vectors were fed into an LSTM with 512 cells in the hidden layer. Output activation sigmoid. The output at each time step is a vector with the same number of entries as there are notes. Each entry is the probability of that note being played at the next time step, conditioned on all prior time steps' inputs.  
![Input/Output Vectors](https://i.imgur.com/FzzR86b.png)

**Training**  
The polyphonic LSTM was trained to reduce cross entropy loss between the outputs and the ground truth. They used Adam with an initial learning rate of $10^{-6}$. They also only trained on 10,000 songs for 4 epochs to reduce training time.

**Generation**  
To generate a new song, they feed a seed of the piano roll and corresponding chords into the LSTM. The notes to be played at the next time step are each sampled independently from the output vector. Because the initial data consisted of multiple instruments playing simultaneously, they implement a method to limit the number of notes to play at any one time step.  
![Music Generation Process](https://i.imgur.com/CvJ7uUq.png)

To do this, they sum all probabilities of the output vector. If it's greater than some limit, they divide all probabilities by the sum and multiply them by that limit.  
$s=\text{sum}\{y_\text{poly}^\text{t}\}=\sum_{i=1}^N P(n_i=1|x_{poly}^0, ... , x_{poly}^{t-1})$.  
${y_\text{poly}^\text{t}}_{new}=y_\text{poly}^\text{t}*(l/s)$.

The piano roll doesn't distinguish a held note or  repeated note of the same pitch, and thus the authors chose to interpret such cases as held notes. Thus consecutive notes of identical pitch are merged, but the beginning of each bar re-strikes all notes. The playback instrumentation and tempo were left to be chosen arbitrarily.

## 5. Results

### Chord LSTM

Their chord LSTM learned some awesome embeddings from its training data. They reduce the ten-dimensional embeddings of the chords to two dimensions using PCA. The result shows the circle of fifths, which is cool cool. They differentiate this result from previous work which was given the circle of fifths for free.

They perform a similar operation on the learned chord embeddings in the shifted dataset. Chords with similar notes were near each other, but there's no circle of fifths to be found due to the C-major shifted dataset.  
![PCA Mappings](https://i.imgur.com/LL20GPp.png)

The chord progressions produced by the LSTM contain structures common in western music, with 4-chord repetitions and many of the most common Top 10 chords. Temperature could be used to increase/decrease the diversity of the chords, with higher temperatures causing increasingly erratic results.

### Polyphonic LSTM

The songs generated contained long term structure with distinct chord changes. There were short sections with dissonance, which could either add suspense or sound wrong. These were bound to happen, as random sampling works like that. A lower temperature could again create stable, boring music, while a high one added harmonic incoherency and diversity.

## 6. Conclusion and Future Work

This work introduces JamBot, a system which predicts chord progressions and generates polyphonic music for those progressions. This generated music had long term structure. The model was able to learn the circle of fifths and related chords purely from the dataset. JamBot chooses what notes to play by sampling from an output vector of probabilities for each note in a piano roll. This was improved by giving the polyphonic LSTM the current chord at each time step.

They note that existing data representations only address held/repeated notes in the monophonic problem space. They also ponder another level to their system's hierarchy which might guide verse/chorus/bridge types of compositional structures. They also bring up the Lakh MIDI dataset's meta information of artists, genres, and lyrics. A "genre feature" might also aid more acute compositions.  
*-Fin 1/22/19-*

# Source 9: https://pdfs.semanticscholar.org/8308/f72cc0c03d48a202aa72f6050600a33b55c2.pdf

http://webprojects.eecs.qmul.ac.uk/marcusp/papers/PearceEtAl2001.pdf